# Fine Tune for Credit Risk

In [3]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import openai
import json

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
import pandas as pd
import json

# read in csv file
df = pd.read_csv('credit_risk_dataset.csv')

# find features with missing values or NaN
print(df.isnull().sum())

# clean the data replace NaN with -99.0
df = df.fillna(-99.0)
#re-order the columns so loan_status is last
cols = list(df.columns.values)
cols.pop(cols.index('loan_status'))
df = df[cols+['loan_status']]
# replace loan_status values 0 with paid and 1 with default
df['loan_status'] = df['loan_status'].replace(0, 'low risk')
df['loan_status'] = df['loan_status'].replace(1, 'high risk')


# print out the first 10 rows
mini_df = df[:100]





person_age                       0
person_income                    0
person_home_ownership            0
person_emp_length              895
loan_intent                      0
loan_grade                       0
loan_amnt                        0
loan_int_rate                 3116
loan_status                      0
loan_percent_income              0
cb_person_default_on_file        0
cb_person_cred_hist_length       0
dtype: int64


In [5]:
# create jsonl file with {"prompt": "data ->", " completion": "loan_status\n\n"} where data is all colums except loan_status
# and loan_status is the value of the loan_status column
# this will be used as the prompt for the GPT-3 model
tdata = []
with open('credit_risk_dataset.jsonl', 'w') as f:
    for index, row in mini_df.iterrows():
        loan_status = row['loan_status']
        # drop the loan_status column
        row = row.drop('loan_status')
        # convert row to dictionary
        row = row.to_dict()
        # convert dictionary to string
        data = json.dumps(row)
        # remove curly braces
        data = data.replace('{', '')
        data = data.replace('}', '')
        data = data.replace('"', '')
    
        
   
        # create the prompt with newline and comma
        prompt = '{"prompt": " ' + data + '->", "completion": " ' + loan_status + '"}'
        # create an array of the prompt
        tdata.append(prompt)
    print(len(tdata))
        

    # write tdata to a json file with newline and comma after each prompt
    with open('credit_risk_dataset.json', 'w') as f:
        for prompt in tdata:
            f.write(prompt+', \n')  

100


In [65]:
train = [{"prompt": " person_age: 22, person_income: 59000, person_home_ownership: RENT, person_emp_length: 123.0, loan_intent: PERSONAL, loan_grade: D, loan_amnt: 35000, loan_int_rate: 16.02, loan_percent_income: 0.59, cb_person_default_on_file: Y, cb_person_cred_hist_length: 3->", "completion": " high risk"}, 
{"prompt": " person_age: 21, person_income: 9600, person_home_ownership: OWN, person_emp_length: 5.0, loan_intent: EDUCATION, loan_grade: B, loan_amnt: 1000, loan_int_rate: 11.14, loan_percent_income: 0.1, cb_person_default_on_file: N, cb_person_cred_hist_length: 2->", "completion": " low risk"}, 
{"prompt": " person_age: 25, person_income: 9600, person_home_ownership: MORTGAGE, person_emp_length: 1.0, loan_intent: MEDICAL, loan_grade: C, loan_amnt: 5500, loan_int_rate: 12.87, loan_percent_income: 0.57, cb_person_default_on_file: N, cb_person_cred_hist_length: 3->", "completion": " high risk"}, 
{"prompt": " person_age: 23, person_income: 65500, person_home_ownership: RENT, person_emp_length: 4.0, loan_intent: MEDICAL, loan_grade: C, loan_amnt: 35000, loan_int_rate: 15.23, loan_percent_income: 0.53, cb_person_default_on_file: N, cb_person_cred_hist_length: 2->", "completion": " high risk"}, 
{"prompt": " person_age: 24, person_income: 54400, person_home_ownership: RENT, person_emp_length: 8.0, loan_intent: MEDICAL, loan_grade: C, loan_amnt: 35000, loan_int_rate: 14.27, loan_percent_income: 0.55, cb_person_default_on_file: Y, cb_person_cred_hist_length: 4->", "completion": " high risk"}, 
{"prompt": " person_age: 21, person_income: 9900, person_home_ownership: OWN, person_emp_length: 2.0, loan_intent: VENTURE, loan_grade: A, loan_amnt: 2500, loan_int_rate: 7.14, loan_percent_income: 0.25, cb_person_default_on_file: N, cb_person_cred_hist_length: 2->", "completion": " high risk"}, 
{"prompt": " person_age: 26, person_income: 77100, person_home_ownership: RENT, person_emp_length: 8.0, loan_intent: EDUCATION, loan_grade: B, loan_amnt: 35000, loan_int_rate: 12.42, loan_percent_income: 0.45, cb_person_default_on_file: N, cb_person_cred_hist_length: 3->", "completion": " high risk"}, 
{"prompt": " person_age: 24, person_income: 78956, person_home_ownership: RENT, person_emp_length: 5.0, loan_intent: MEDICAL, loan_grade: B, loan_amnt: 35000, loan_int_rate: 11.11, loan_percent_income: 0.44, cb_person_default_on_file: N, cb_person_cred_hist_length: 4->", "completion": " high risk"}, 
{"prompt": " person_age: 24, person_income: 83000, person_home_ownership: RENT, person_emp_length: 8.0, loan_intent: PERSONAL, loan_grade: A, loan_amnt: 35000, loan_int_rate: 8.9, loan_percent_income: 0.42, cb_person_default_on_file: N, cb_person_cred_hist_length: 2->", "completion": " high risk"}, 
{"prompt": " person_age: 21, person_income: 10000, person_home_ownership: OWN, person_emp_length: 6.0, loan_intent: VENTURE, loan_grade: D, loan_amnt: 1600, loan_int_rate: 14.74, loan_percent_income: 0.16, cb_person_default_on_file: N, cb_person_cred_hist_length: 3->", "completion": " high risk"}]

In [66]:
def prepare_data(dictionary_data, final_file_name):
    with open(final_file_name, 'w') as outfile:
        for entry in dictionary_data:
            json.dump(entry, outfile)
            outfile.write('\n')


In [67]:
prepare_data(train, 'john.jsonl')

In [68]:
!openai tools fine_tunes.prepare_data -f "john.jsonl"

4455.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Analyzing...

- Your file contains 10 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- Based on your data it seems like you're trying to fine-tune a model for classification
- For classification, we recommend you try one of the faster and cheaper models, such as `ada`
- For classification, you can estimate the expected model performance by keeping a held out dataset, which is not used for training
- All prompts end with suffix `->`
- All prompts start with prefix ` person_age: 2`. Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion

Based on the analysis we will perform the following actions:
- [Recommended] Remove prefix ` person_age: 2` from all prompts [Y/n]: ^C



In [73]:
import openai

# set the OpenAI API key
openai.api_key = os.environ.get('OPENAI_API_KEY')

# read the training data from the file
with open('john.jsonl', 'r') as f:
    training_data = f.read()

# create the fine-tuning request
response = openai.Completion.create(
    engine="ada",
    prompt=training_data,
    temperature=0.5,
    max_tokens=200,
    n=1,
    stop=None,
    frequency_penalty=0,
    presence_penalty=0
)

# print the response
print(response)




{
  "id": "cmpl-8HEGAK8ha4zINHlDYZdsi1Q9xxUPs",
  "object": "text_completion",
  "created": 1699117082,
  "model": "ada",
  "choices": [
    {
      "text": "\n{\"prompt\": \" person_age: 25, person_income: 9600, person_home_ownership: MORTGAGE, person_emp_length: 1.0, loan_intent: MEDICAL, loan_grade: C, loan_amnt: 5500, loan_int_rate: 12.87, loan_percent_income: 0.57, cb_person_default_on_file: N, cb_person_cred_hist_length: 3->\", \"completion\": \" high risk\"}\n\n{\"prompt\": \" person_age: 21, person_income: 9600, person_home_ownership: OWN, person_emp_length: 2.0, loan_intent: VENTURE, loan_grade: A, loan_amnt: 2500, loan_int_rate: 7.14, loan_percent_income: 0.25, cb",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 1160,
    "completion_tokens": 200,
    "total_tokens": 1360
  }
}
